In [1]:
#ch12_ALS_52a_win_20180506.ipynb, 
#Collaborative Filtering- spark.ml, 
#https://spark.apache.org/docs/2.0.0-preview/ml-collaborative-filtering.html
#PySparkWin

In [2]:
#Collaborative filtering is commonly used for recommender systems. 
#spark.ml currently supports model-based collaborative filtering, in which users and products are described by a small set of latent factors 
#that can be used to predict missing entries.
#spark.ml uses the alternating least squares (ALS) algorithm to learn these latent factors.

#https://spark.apache.org/docs/2.0.0-preview/api/python/pyspark.ml.html#pyspark.ml.recommendation.ALS

In [3]:
#The implementation in spark.ml has the following parameters:
#numBlocks --> is the number of blocks the users and items will be partitioned into in order to parallelize computation (defaults to 10).
#           rank --> is the number of latent factors in the model (defaults to 10).
#     maxIter --> is the maximum number of iterations to run (defaults to 10).
#  regParam --> specifies the regularization parameter in ALS (defaults to 1.0).
#implicitPrefs --> specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data
#                          (defaults to false which means using explicit feedback).
#        alpha  --> is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations 
#                         (defaults to 1.0).
#nonnegative --> specifies whether or not to use nonnegative constraints for least squares (defaults to false).

In [4]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [5]:
#1. 準備資料

In [6]:
#from pyspark.ml import modules:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [3]:
#create SparkSession "spark01"
#from pyspark.sql import SparkSession
#spark01 = SparkSession \
#    .builder \
#    .appName("Python Spark SQL basic example") \
#    .config("spark.some.config.option", "some-value") \
#    .getOrCreate()

In [47]:
#header=['userId','movieId','rating','timestamp']
df=spark.read.format('csv').load('u.csv')

In [48]:
df.show(5)

+---+---+---+---------+
|_c0|_c1|_c2|      _c3|
+---+---+---+---------+
|  0|  1|  2|        3|
|196|242|  3|881250949|
|186|302|  3|891717742|
| 22|377|  1|878887116|
|244| 51|  2|880606923|
+---+---+---+---------+
only showing top 5 rows



In [49]:
spark.read.format?

In [12]:
#以 SparkSession.read.text() 讀取 text file, 轉成DataFrame "rawUserDataDF"
rawUserDataDF=spark.read.text("u.data")

In [13]:
rawUserDataDF.printSchema()

root
 |-- value: string (nullable = true)



In [8]:
rawUserDataDF.first()

Row(value='196\t242\t3\t881250949')

In [9]:
#以 DataFrame.rdd 將 DataFrame "rawUserDataDF" 轉成RDD "rawDataRDD"
rawDataRDD=rawUserDataDF.rdd

In [10]:
rawDataRDD.first() #檢視 rawDataRDD 第一筆資料

Row(value='196\t242\t3\t881250949')

In [11]:
#以RDD.map(lambda) 將 RDD "rawDataRDD" 依 "::" 分割 map 成 RDD "partsRDD"
partsRDD = rawDataRDD.map(lambda r: r.value.split("\t"))

In [12]:
partsRDD.first() 

['196', '242', '3', '881250949']

In [13]:
#資料型別轉換, 使用 Row type
ratingsRDD = partsRDD.map(lambda x: Row(userId=int(x[0]), movieId=int(x[1]),
                                     rating=float(x[2]), timestamp=long(x[3])))

In [14]:
ratingsRDD.first()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\spark-2.2.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 177, in main
  File "C:\spark\spark-2.2.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 172, in process
  File "C:\spark\spark-2.2.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\spark\spark-2.2.1-bin-hadoop2.7\python\pyspark\rdd.py", line 1339, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-13-34783c8bb7da>", line 3, in <lambda>
NameError: name 'long' is not defined

	at org.apache.spark.api.python.Pyt

In [16]:
#最後,再以SparkSeeeion.createDataFrame(RDD), 將RDD "ratingsRDD" 轉回成 DataFrame "ratingsDF"
ratingsDF = spark.createDataFrame(ratingsRDD)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\spark-2.2.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 177, in main
  File "C:\spark\spark-2.2.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 172, in process
  File "C:\spark\spark-2.2.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\spark\spark-2.2.1-bin-hadoop2.7\python\pyspark\rdd.py", line 1339, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-13-34783c8bb7da>", line 3, in <lambda>
NameError: name 'long' is not defined

	at org.apache.spark.api.python.Pyt

In [22]:
ratingsDF.show(5)

+-------+------+---------+------+
|movieId|rating|timestamp|userId|
+-------+------+---------+------+
|    242|   3.0|881250949|   196|
|    302|   3.0|891717742|   186|
|    377|   1.0|878887116|    22|
|     51|   2.0|880606923|   244|
|    346|   1.0|886397596|   166|
+-------+------+---------+------+
only showing top 5 rows



In [23]:
#於是,我們可以用 DataFrame-Based Collaborative filtering 運算

In [35]:
#step 1: 以DataFrame.randomSplit() 將 DataFrame "ratingsDF" 依設定機率比例 (8:2) 分成訓練資料 "trainingDF" 及 測試資料 "testDF"
(trainingDF, testDF) = ratingsDF.randomSplit([0.8, 0.2])

In [37]:
print('訓練資料筆數: '+str(trainingDF.count())+' 測試資料筆數: '+str(testDF.count()))

訓練資料筆數: 1187 測試資料筆數: 314


In [38]:
#step 2:  Build the recommendation model  " alsModel" using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating") #als, Alternating Least Squares (ALS) matrix factorization.

In [41]:
#以 Estimator.fit(training Data), 訓練階段, 得到模型 alsModel
alsModel=als.fit(trainingDF)  #alsModel, pyspark.ml.recommendation.ALSModel

In [43]:
type(alsModel)

pyspark.ml.recommendation.ALSModel

In [44]:
#step 3:  Evaluate the model by computing the RMSE on the test data
#             We evaluate the recommendation model by measuring the root-mean-square error of rating prediction.

In [45]:
#使用 Transformer.transform(test data), 預測test 資料 "testDF", 回傳的是 DataFrame "predictionsDF"
predictionsDF = alsModel.transform(testDF)

In [46]:
predictionsDF.show(5)

+-------+------+----------+------+----------+
|movieId|rating| timestamp|userId|prediction|
+-------+------+----------+------+----------+
|     31|   3.0|1424380312|     7|0.02137813|
|     31|   1.0|1424380312|    18| 1.6260848|
|     85|   1.0|1424380312|    12| 2.9139917|
|     85|   3.0|1424380312|     6| 2.6702542|
|     85|   5.0|1424380312|     8| 5.3887396|
+-------+------+----------+------+----------+
only showing top 5 rows



In [47]:
#以 RegressionEvaluator() 建立評估器 "evaluator01", pyspark.ml.evaluation.RegressionEvaluator
evaluator01 = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

In [49]:
#以 RegressionEvaluator.evaluate() 計算 rmse
rmse = evaluator01.evaluate(predictionsDF)

In [51]:
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.89281881552
